In [3]:
# %% [1] 安裝依賴 & 導入庫
!pip install psycopg2-binary matplotlib pandas --quiet




[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip


In [23]:

import psycopg2
import time
import pandas as pd
import matplotlib.pyplot as plt

# 配置數據庫連接 (根據你的環境修改)
DB_CONFIG = {
    "host": "localhost",
    "database": "postgres",
    "user": "postgres",
    "password": "postgres",
    "port": 5432
}

# 建立連接
conn = psycopg2.connect(**DB_CONFIG)
conn.autocommit = True
cursor = conn.cursor()

In [24]:
# %% [2] 初始化測試表
def init_test_table():
    try:
        cursor.execute("""
            DROP TABLE IF EXISTS test_fragmentation;
            CREATE TABLE test_fragmentation (
                id SERIAL PRIMARY KEY,
                random_data TEXT,
                created_at TIMESTAMP DEFAULT NOW()
            );
            ALTER TABLE test_fragmentation SET (autovacuum_enabled = off);
        """)
        print("✅ 測試表創建完成，autovacuum 已禁用")
        
        # 插入初始數據
        cursor.execute("""
            INSERT INTO test_fragmentation (random_data)
            SELECT md5(random()::text)
            FROM generate_series(1, 1000000)
        """)
        print(f"📥 已插入 {cursor.rowcount} 行初始數據")
        
    except Exception as e:
        print(f"❌ 初始化錯誤: {str(e)}")

init_test_table()

✅ 測試表創建完成，autovacuum 已禁用
📥 已插入 1000000 行初始數據


In [26]:
# %% [3] 製造碎片化
def create_fragmentation(cycles=10):
    try:
        for cycle in range(1, cycles+1):
            start = time.time()
            
            # 更新操作
            cursor.execute("""
                UPDATE test_fragmentation
                SET random_data = md5(random()::text)
                WHERE random() < 0.5
            """)
            updated = cursor.rowcount
            
            # 刪除操作
            cursor.execute("""
                DELETE FROM test_fragmentation
                WHERE random() < 0.2
            """)
            deleted = cursor.rowcount
            
            # 插入操作
            cursor.execute("""
                INSERT INTO test_fragmentation (random_data)
                SELECT md5(random()::text)
                FROM generate_series(1, 200000)
            """)
            
            print(f"♻️ 週期 {cycle}/{cycles} | "
                  f"更新: {updated}, 刪除: {deleted}, 插入: {200000} | "
                  f"耗時: {time.time()-start:.1f}s")
            
    except Exception as e:
        print(f"❌ 碎片化操作錯誤: {str(e)}")

create_fragmentation(cycles=10)

♻️ 週期 1/10 | 更新: 98102, 刪除: 39514, 插入: 200000 | 耗時: 1.3s
♻️ 週期 2/10 | 更新: 178022, 刪除: 70881, 插入: 200000 | 耗時: 1.9s
♻️ 週期 3/10 | 更新: 242497, 刪除: 97044, 插入: 200000 | 耗時: 2.1s
♻️ 週期 4/10 | 更新: 294313, 刪除: 117860, 插入: 200000 | 耗時: 3.3s
♻️ 週期 5/10 | 更新: 335783, 刪除: 134734, 插入: 200000 | 耗時: 3.4s
♻️ 週期 6/10 | 更新: 367568, 刪除: 147134, 插入: 200000 | 耗時: 3.3s
♻️ 週期 7/10 | 更新: 394356, 刪除: 157221, 插入: 200000 | 耗時: 3.1s
♻️ 週期 8/10 | 更新: 415219, 刪除: 165758, 插入: 200000 | 耗時: 5.3s
♻️ 週期 9/10 | 更新: 432903, 刪除: 173368, 插入: 200000 | 耗時: 3.2s
♻️ 週期 10/10 | 更新: 446868, 刪除: 178552, 插入: 200000 | 耗時: 3.1s


In [27]:
# %% [4] 執行 VACUUM
cursor.execute("VACUUM (DISABLE_PAGE_SKIPPING, VERBOSE) test_fragmentation")
print("🧹 VACUUM 完成")

🧹 VACUUM 完成


In [28]:
# %% [5] 修正版碎片化分析函數
def analyze_fragmentation():
    try:
        # 獲取索引統計
        cursor.execute("""
            SELECT 
                pg_size_pretty(pg_relation_size('test_fragmentation_pkey'::regclass)),
                idx_scan,
                idx_tup_read,
                idx_tup_fetch
            FROM pg_stat_all_indexes
            WHERE indexrelid = 'test_fragmentation_pkey'::regclass
        """)
        stats = cursor.fetchone()
        
        # 獲取頁面元數據（修正类型转换和字段）
        cursor.execute("""
            SELECT 
                level as btree_depth,
                root as root_page,
                pg_size_pretty(pg_relation_size('test_fragmentation_pkey')) AS index_size_mb
            FROM bt_metap('test_fragmentation_pkey')
        """)
        page_stats = cursor.fetchone()

        return {
            # 基础统计
            "index_size": stats[0],
            "index_scans": stats[1],
            "tuples_read": stats[2],
            "tuples_fetched": stats[3],
            
            # B-Tree 结构元数据
            "btree_depth": page_stats[0],
            "root_page": page_stats[1],  # 修正字段名
            "index_size_mb": page_stats[2],
            
            # 新增重要指标
            "avg_fanout": calculate_fanout(page_stats[0])  # 添加B-Tree扇出计算
        }
        
    except Exception as e:
        print(f"分析错误: {str(e)}")
        return None

# 新增辅助函数
def calculate_fanout(btree_depth):
    """计算B-Tree平均扇出系数"""
    if btree_depth < 2:
        return None
    
    cursor.execute("""
        SELECT 
            pg_relpages('test_fragmentation_pkey'::regclass)::float,
            reltuples
        FROM pg_class
        WHERE relname = 'test_fragmentation_pkey'
    """)
    total_pages, total_tuples = cursor.fetchone()
    return round((total_tuples ** (1/btree_depth)) / 1000)  # 以千为单位

# 第一次分析
fragmented = analyze_fragmentation()
df_frag = pd.DataFrame([fragmented]).T.rename(columns={0: '碎片化後'})
df_frag

,碎片化後
index_size,133 MB
index_scans,0
tuples_read,0
tuples_fetched,0
btree_depth,2
root_page,412
index_size_mb,133 MB
avg_fanout,1


In [29]:
# %% [6] 執行 REINDEX
def perform_reindex():
    start = time.time()
    cursor.execute("REINDEX INDEX test_fragmentation_pkey")
    print(f"🔄 REINDEX 完成 | 耗時: {time.time()-start:.1f}s")

perform_reindex()

🔄 REINDEX 完成 | 耗時: 0.4s


In [30]:
# %% [7] REINDEX 後分析
reindexed = analyze_fragmentation()

# 合併比較數據
df_compare = pd.DataFrame([fragmented, reindexed], index=['碎片化後', 'REINDEX後'])
df_compare.T

,碎片化後,REINDEX後
index_size,133 MB,20 MB
index_scans,0,0
tuples_read,0,0
tuples_fetched,0,0
btree_depth,2,2
root_page,412,290
index_size_mb,133 MB,20 MB
avg_fanout,1,1


In [31]:
# %% [9] 清理環境
cursor.execute("ALTER TABLE test_fragmentation SET (autovacuum_enabled = on)")
cursor.close()
conn.close()
print("🧼 數據庫連接已關閉，autovacuum 已啟用")

🧼 數據庫連接已關閉，autovacuum 已啟用
